In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime
import uuid
import requests
import json
from azure.cosmos.exceptions import CosmosResourceNotFoundError
from datetime import datetime, timedelta
from whatsapp_apis.whatsapp import WhatsappAPI
from whatsapp_apis.llm import GroqModel
from whatsapp_apis.cosmos_database import CosmoDataBase
from pytz import timezone
import pytz
from datetime import datetime, timedelta
import logging

load_dotenv()

class PostWebhook:

    def __init__(self):
        self.db_container_user = os.getenv('COSMOS_DB_CONTAINER_USER')
        self.db_container_chat_history = os.getenv('COSMOS_DB_CONTAINER_CHAT_HISTORY')
        self.access_token = os.getenv('PERMANENT_TOKEN_FACEBOOK')
        self.code_phone_from = os.getenv('CODE_PHONE_FROM')
        self.cosmos_db = CosmoDataBase() 
        
    def main(self):
        

        body = {}
        recipient_id = '56999319606'
        message_id_ = 'sfjdsfjfjdsfjssadsdsafdjfjdsf'
        import time
        # Get the current Unix timestamp
        timestamp_ = int(time.time())              
        text_ = 'cual es mi nombre'
        type_ = 'text'
        metadata_ = ''
        json_response = {'from': recipient_id, 'message_id': message_id_, 'timestamp': timestamp_, 'text': text_, 'type': type_, 'metadata': metadata_, 'typeuser': 'user'}
        
        # Paso 2 - Detectar si existe una conversacion anterior en Cosmos DB, sino  crearla.    
        response_detection_cosmo = self.cosmos_db.get_save_cosmos_db_history_chat(self.db_container_chat_history, json_response)
        history_ = response_detection_cosmo['history']

        # Paso 3 - Genearar una respuesta del modelo LLM.   
        response_llm_modelo = GroqModel(history_).main()['message']
        print(history_)
        current_timestamp = datetime.utcnow().isoformat()
        json_response_assistant = {'from': recipient_id, 'message_id': message_id_, 'timestamp': current_timestamp, 'text': response_llm_modelo, 'type': type_, 'metadata': {}, 'typeuser': 'assistant'}
        response_db_assitante = self.cosmos_db.get_save_cosmos_db_history_chat(self.db_container_chat_history, json_response_assistant)
        print(response_db_assitante['replacted'])
        print(self.code_phone_from)
        print(response_llm_modelo)
        print(self.access_token)
        if response_db_assitante['replacted'] == False:
            
            # Paso 4 - Enviar respuesta al usuario.  
            response_send_message = WhatsappAPI(body).text_message(self.access_token, recipient_id, self.code_phone_from, response_llm_modelo)
            print(response_send_message)
            return json.dumps({'status_code': 200, 'response': 'Message sent'})
        else:
            return json.dumps({'status_code': 200, 'response': 'Message already sent'})
    


response = PostWebhook().main()
response

[{'role': 'user', 'content': 'Hola', 'timestamp': '2024-07-14T16:34:30.146629', 'message_id_': 'wamid.HBgLNTY5OTkzMTk2MDYVAgASGCAyRDcxNjI2QTVGOUIxOUZCQjQ3NDM2ODJFRUM2OUQwOQA=', 'type': 'text'}, {'role': 'assistant', 'content': '¡Hola! ¿En qué puedo ayudarte hoy? Estoy aquí para ayudarte con cualquier pregunta o tarea que tengas.', 'timestamp': '2024-07-14T16:34:31.382639', 'message_id_': 'wamid.HBgLNTY5OTkzMTk2MDYVAgASGCAyRDcxNjI2QTVGOUIxOUZCQjQ3NDM2ODJFRUM2OUQwOQA='}, {'role': 'user', 'content': 'Me llamo Francisco, cual es mi nombre', 'timestamp': '2024-07-14T16:35:09.028746', 'message_id_': 'wamid.HBgLNTY5OTkzMTk2MDYVAgASGCBCMkY2N0FENTQ0MzI3MzczRkE3QTEzOTc0QTU1MjdCQQA='}, {'role': 'assistant', 'content': '¡Hola! ¿En qué puedo ayudarte hoy? Estoy aquí para servirle y responder a todas sus preguntas. ¡Sentirme útil es mi objetivo!', 'timestamp': '2024-07-14T16:35:09.812386', 'message_id_': 'wamid.HBgLNTY5OTkzMTk2MDYVAgASGCBCMkY2N0FENTQ0MzI3MzczRkE3QTEzOTc0QTU1MjdCQQA='}, {'role': 'use

'{"status_code": 200, "response": "Message sent"}'

In [9]:
body_ = {
        "object": "whatsapp_business_account",
        "entry": [
            {
                "id": "380441975145057",
                "changes": [
                    {
                        "value": {
                            "messaging_product": "whatsapp",
                            "metadata": {
                                "display_phone_number": "56958381267",
                                "phone_number_id": "336050666261796"
                            },
                            "contacts": [
                                {
                                    "profile": {
                                        "name": "Foco"
                                    },
                                    "wa_id": "56999319606"
                                }
                            ],
                            "messages": [
                                {
                                    "from": "56999319606",
                                    "id": "wamid.HBgLNTY5OTkzMTk2MDYVAgASGCA3NUFCNEVDOURDNDkwMEUzNEJDOEQ3M0FENTEyMzQ1NwA=",
                                    "timestamp": "1719552010",
                                    "text": {
                                        "body": "cual es mi nombre"
                                    },
                                    "type": "text"
                                }
                            ]
                        },
                        "field": "messages"
                    }
                ]
            }
        ]
}

from whatsapp_apis import post_webhook

post_webhook_instance = post_webhook.PostWebhook(body_)
result = post_webhook_instance.main()

In [10]:
result

'{"status_code": 200, "response": "Message sent"}'